# Considerations prior to the modelling process

Before going further in the modelling process, it seems necessary to speak a bit about its implications, which may even invalidate the approach altogether.

It's good to recall that, often, machine learning is mistaken as a perfect solution for every problem. When in doubt, we can refer to the [Rule \#1 by Google](https://developers.google.com/machine-learning/guides/rules-of-ml/#rule_1_don%E2%80%99t_be_afraid_to_launch_a_product_without_machine_learning): ***Don’t be afraid to launch a product without machine learning.***

So, what are we really doing in this case? We have a dataset with some real estate information (address, \#rooms, \#parking spots) and a price estimate for a unit in the building. The main consideration here is that if we really want to build an AVM upon this data alone we would have to use the point estimate as the label. But the point estimate is also an estimate built on top of other datasets (probably some sort of average of distinct units for sale at a given time). As with every approximate model, there are assumptions taken and results that in the best case scenario only approximate the real price function. Building a model on top of that estimates would propagate the error to a higher degree, as the goal would become approximating the approximation (that we know nothing about its nature, but gives a ~26% uncertainty range), instead of the actual transaction price of the real estate.

In the end, I don't think it's worth continuing with the modelling process for this particular dataset, but I will outline the basic steps:

 1. Choose a metric (and a loss function)
 2. Choose a validation technique
 3. Look for outliers and missing values
 4. Categorical data prep
 5. Choose a machine learning model

### 1. Metric and loss function

This is a regression problem. There is a standard of using the `MSE` (a.k.a. L2 Loss) for every situation but we first have to understand its implications. Since it takes the square of the difference between the target and the predicted value, if there's an outlier then the error becomes very high, skewing the training process and parameters toward the outlier.

As for the metric, we should avoid using the MSE or its variants (like RMSE), and even the MAE because the range of real estate prices spans multiple orders of magnitude. If theres a home valuated at R\\$120.000 while its true price if R\\$100.000, we're talking about a 20% loss, but if there's a home valuated at R\\$1.020.000 when its value is R\\$1.000.000, then it's a 2% loss even though the MSE is the same (and the second prediction is way better). So a good metric would involve a percentual difference between the predicted and the actual price. Taking the full validation set to calculate a single metric from, we can opt for a `mean` or even a `median` to minimize the effect of outliers.

In [ ]:
import numpy
def median_percent_error(y_true, y_pred):
    return numpy.median((y_true - y_pred) / y_true)

### 2. Validation technique

Given a static dataset as we have, we could really go with a random 80/20 split. If we had more locations (as in the original dataset) the advisable approach would to use a Stratified K-Fold validation. The downsides of the random split is that it assumes the data will not change distribution for future predictions (or even that the training data accurately portrays the original population while being only a sample), which is almost never true.

### 3. Outliers and missing values

Depending on the model we choose to use, there's actually no need to look into outliers very thoroughly. As the data already has imputations for missing values (`-1`), we will postpone the discussion of outliers and missing values to when we choose our model.

### 4. Categorical data preparation

For categorical data, there are some interesting approaches to be considered, depending on the cardinality. For low cardinality features like `building_type`, an One-Hot encoding approach may be a good idea. Since the only `city` remaining is `São Paulo`, we can disregard this feature, along with `state`.

As we saw in the previous notebook, `tower_name` has low or even zero prediction power since for many different towers in the same condominium, the point estimate per sqm is the same.

As for the `address`, they're to an extent encoded in the geolocation values, but not totally. An estimate can be heavily influenced by the name of the street/ave it's located on. But since the address has very high cardinality, a better way to handle these variables would be to couple this information with a demand proxy and generate semantic related multidimensional embeddings for each street. The same can be done for geolocations encoded with GeoHash or S2 cells, and it's an approach used by companies like AirBnb to assess the characteristics of a specific region.

### 5. The ML Model

To choose a good machine learning model for the task we can rely on two factors: the interpretability and accuracy. We can get some perks for free (e.g. robustness to outliers) depending on the choice we make.

Interpretability is a good thing to have when dealing with real estate. As it's a traditional industry and the unit prices are the highest compared to any other industry, a commitment in money must be grounded in a solid reasoning. It's hard to convince stakeholders when a machine learning model gives a lower estimate for a home with swimming a pool than another, with identical features, but without a pool.

Most of well known AVMs use tree based model (like Zillow's Zestimate). While the accuracy is higher by using them, it comes at the cost of interpretability, as the decision boundaries of ensemble tree models aren't smooth and are prone to situations like the above. On the other hand, those models are also robust to outliers and don't require normalization making them a good choice for the task.